# NLP PROJECT

## RIDDHI MEHTA - J030
## HUSAIN GHADIALI - J056

### IMPORTING LIBRARIES

In [2]:
import pandas as pd
import numpy as np

import re
# import pickle
import json
import io

from io import StringIO

# import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import Normalize
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

# import pydotplus
# import pydot

import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.multioutput import MultiOutputClassifier
# from sklearn.model_selection import train_test_split
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.metrics import accuracy_score
# from sklearn.multiclass import OneVsRestClassifier
# from sklearn.preprocessing import StandardScaler
# from sklearn.pipeline import Pipeline


from tensorflow.python.keras.layers import Dense, Embedding, Input, LSTM, Bidirectional, GlobalMaxPool1D, Dropout, Conv1D, MaxPooling1D, GRU
from tensorflow.python.keras import Sequential
from tensorflow.python.keras.models import Model, load_model
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
# from tensorflow.python.keras_preprocessing.text import tokenizer_from_json

# import keras
from keras_preprocessing.text import tokenizer_from_json

# from tensorflow.python.keras.layers import Dense, Embedding, Input, LSTM, Bidirectional, GlobalMaxPool1D, Dropout
# from keras.callbacks import EarlyStopping, ModelCheckpoint


nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

import warnings
warnings.filterwarnings("ignore")

import random
from tkinter import *

C:\Users\Riddhi\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Riddhi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Riddhi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Riddhi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### CREATING THE GRAPHICAL USER INTERFACE USING TKINTER

In [14]:
root = Tk()
root.title("Let's Fight Online Abuse")

# canvas1 = Canvas(root, width = 500, height = 500, background = "skyblue")
# canvas1.grid(row = 0, column = 1)

frame = Frame(root)
frame.grid(row = 0, column = 0, sticky = "n", pady = 30)

label_title = Label(frame, text = "Please enter your comment below")
label_title.grid(row = 0, column = 0, sticky = "n", pady = 30)

entry1 = Text(frame, width = 20, height = 10, bg = "white", cursor = "dot", wrap = WORD) 
entry1.grid(row = 1, column = 0, sticky = "n", pady = 30)

def predict_my_sentence():
    
    ## Preprocessing

    text = entry1.get("1.0", END)
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text) # substitutes non word character with a space
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    text = re.sub(r"[0-9]"," ",text) # substitutes numbers with a blank space
    
    ## Removal of stop words

    my_tokens = list() # Initialising a list of tokens that will be formed
    stop_words= set(stopwords.words('english')) # Extracting english stopwords from nltk corpus

    tokens = word_tokenize(text, preserve_line=False)

    for token in tokens:
        if (token not in stop_words):
            my_tokens.append(token)
            
    ## Joining all tokens into one complete sentence

    final = ' '.join([str(elem) for elem in my_tokens]) # makes it an entire string
    
    ## Converting the sentence into a suitable datatype

    test = pd.DataFrame({'comment_text':final}, index = [0])
    
    ## Loading saved tokenizer to fit into test tokens

    with open(r"D:\RIDDHI MEHTA\MPSTME\Third Year\Semester VI\Natural Language Processing\Project\Toxic Comments\tokenizer.json") as f:
        data = json.load(f)
        tokenizer = tokenizer_from_json(data)

    test = tokenizer.texts_to_sequences(test)
    
    ## Padding sequences

    maxlen = 400
    test = pad_sequences(test, maxlen = maxlen, padding = "post")
    
    ## Loading neural network model for prediction

    model = load_model(r"D:\RIDDHI MEHTA\MPSTME\Third Year\Semester VI\Natural Language Processing\Project\Toxic Comments\nn1_new.hdf5")

    pred = model.predict(test, verbose = 1)
    pred = list(pred[0])
    
    ## Creating a dictionary of labels and its respective probabilities 
    
    labels = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
    
    result = dict(zip(labels, pred))
    
    ## Finding out the main classes of the sentence and printing them
    
    outer_list = []
    
    for keys, values in result.items():
        if values >= 0.1:
            outer_list.append(keys)
        else:
            continue
            
    # Plotting heat intensity bar plot of every label probability
    
    my_cmap = cm.get_cmap('jet')
    my_norm = Normalize(vmin=0, vmax=1)
    
    y = np.array(pred)
    
    figure = Figure(figsize=(8,5), dpi=100)
    subplot = figure.add_subplot(111) 
    subplot.bar(range(len(y)), y, color = my_cmap(my_norm(pred)))
    subplot.set_title('Toxic Comments Classification')
    subplot.set_xlabel("Type of Comment")
    subplot.set_ylabel("Probability")
    subplot.set_xticks(range(len(y)))
    subplot.set_xticklabels(["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"])
    
#     plt.tick_params(labelcolor="none", bottom=False, left=False)
#     plt.xlabel('Type of Comment', fontsize=10)
#     plt.ylabel('Probability', fontsize=10)
#     plt.xticks([0,1,2,3,4,5], labels, fontsize=10, rotation=30)
    
    canvas = FigureCanvasTkAgg(figure, root)
    canvas.show()
    canvas.get_tk_widget().grid(row = 0, column = 1, pady = 30)
    
    label_output = Label(frame, text = "This sentence can be mainly classifed as " + str(outer_list))
    label_output.grid(row = 3, column = 0, pady = 30)

button1 = Button(frame, text='Predict my sentence', command = predict_my_sentence)
button1.grid(row = 2, column = 0, pady = 30)

root.mainloop()

1/1 [==============================] - 0s 374ms/sample


#### The output of this code is on the last pages of the PDF attached in this folder. Kindly refer that PDF.

# ------------------THE END------------------